# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report, accuracy_score
from sklearn.externals import joblib

import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

/Users/kevinwebb/Desktop/env/ds/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)
/Users/kevinwebb/Desktop/env/ds/lib/python3.7/site-packages/nltk/decorators.py:68: DeprecationWarning: `formatargspec` is deprecated since Python 3.5. Use `signature` and the `Signature` object directly
  regargs, varargs, varkwargs, defaults, formatvalue=lambda value: ""
/Users/kevinwebb/Desktop/env/ds/lib/python3.7/site-packages/nltk/lm/counter.py:15: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collection

True

In [2]:
# load data from database
engine = create_engine('sqlite:///project.db')
df = pd.read_sql_table('project', engine)

X = df.message.values
Y = df[df.columns[4:]].values
category_names = list(df.columns[4:])


### 2. Write a tokenization function to process your text data

In [3]:
def tokenize(text):
    stop_words = stopwords.words("english")
    lemmatizer = WordNetLemmatizer()
    
    # normalize case and remove punctuation
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    # tokenize text
    tokens = word_tokenize(text)
    
    # lemmatize andremove stop words
    tokens = [lemmatizer.lemmatize(word).lower().strip() for word in tokens if word.lower().strip() not in stop_words]

    return tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [ ]:
def model_pipeline():
    pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(OneVsRestClassifier(RandomForestClassifier())))
    ])

    return pipeline

pipeline = model_pipeline()

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, Y)

In [6]:
# train classifier
pipeline.fit(X_train, y_train)

/opt/conda/lib/python3.6/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 0.0 is present in all training examples.
  str(classes[c]))


Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...ore=False, random_state=1, verbose=0, warm_start=False),
          n_jobs=1),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [10]:
y_pred = pipeline.predict(X_test)

In [11]:
print(classification_report(y_test, y_pred, target_names = category_names))

                        precision    recall  f1-score   support

               related       0.66      0.90      0.76      1647
               request       0.42      0.18      0.25       847
                 offer       0.00      0.00      0.00         2
           aid_related       0.45      0.25      0.32       937
          medical_help       0.00      0.00      0.00       131
      medical_products       0.00      0.00      0.00        81
     search_and_rescue       0.00      0.00      0.00        54
              security       0.00      0.00      0.00        29
              military       0.00      0.00      0.00         7
           child_alone       0.00      0.00      0.00         0
                 water       0.00      0.00      0.00       181
                  food       0.22      0.02      0.03       352
               shelter       0.06      0.00      0.01       247
              clothing       0.00      0.00      0.00        25
                 money       0.00      

### 6. Improve your model
Use grid search to find better parameters. 

In [19]:
print(classification_report(y_test, y_pred, target_names = category_names))

                        precision    recall  f1-score   support

               related       0.66      0.90      0.76      1647
               request       0.42      0.18      0.25       847
                 offer       0.00      0.00      0.00         2
           aid_related       0.45      0.25      0.32       937
          medical_help       0.00      0.00      0.00       131
      medical_products       0.00      0.00      0.00        81
     search_and_rescue       0.00      0.00      0.00        54
              security       0.00      0.00      0.00        29
              military       0.00      0.00      0.00         7
           child_alone       0.00      0.00      0.00         0
                 water       0.00      0.00      0.00       181
                  food       0.22      0.02      0.03       352
               shelter       0.06      0.00      0.01       247
              clothing       0.00      0.00      0.00        25
                 money       0.00      

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [15]:
y_pred = cv.predict(X_test)

In [18]:
for i in range(y_test.shape[1]):
        print('%25s accuracy : %.2f' %(category_names[i], accuracy_score(y_test[:,i], y_pred[:,i])))
        
        

                  related accuracy : 0.63
                  request accuracy : 0.64
                    offer accuracy : 1.00
              aid_related accuracy : 0.61
             medical_help accuracy : 0.95
         medical_products accuracy : 0.97
        search_and_rescue accuracy : 0.98
                 security accuracy : 0.99
                 military accuracy : 1.00
              child_alone accuracy : 1.00
                    water accuracy : 0.93
                     food accuracy : 0.86
                  shelter accuracy : 0.90
                 clothing accuracy : 0.99
                    money accuracy : 0.99
           missing_people accuracy : 0.99
                 refugees accuracy : 0.99
                    death accuracy : 0.97
                other_aid accuracy : 0.85
   infrastructure_related accuracy : 0.97
                transport accuracy : 0.98
                buildings accuracy : 0.96
              electricity accuracy : 0.99
                    tools accuracy

In [19]:
print(classification_report(y_test, y_pred, target_names = category_names))

                        precision    recall  f1-score   support

               related       0.66      0.90      0.76      1647
               request       0.42      0.18      0.25       847
                 offer       0.00      0.00      0.00         2
           aid_related       0.45      0.25      0.32       937
          medical_help       0.00      0.00      0.00       131
      medical_products       0.00      0.00      0.00        81
     search_and_rescue       0.00      0.00      0.00        54
              security       0.00      0.00      0.00        29
              military       0.00      0.00      0.00         7
           child_alone       0.00      0.00      0.00         0
                 water       0.00      0.00      0.00       181
                  food       0.22      0.02      0.03       352
               shelter       0.06      0.00      0.01       247
              clothing       0.00      0.00      0.00        25
                 money       0.00      

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [20]:
joblib.dump(cv, "classifier.pkl")

['classifier.pkl']

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.